## Import thư viện

In [ ]:
!pip install pyngrok

In [ ]:
!pip install streamlit

In [ ]:
import os
os.system('pip install /kaggle/input/kerasapplications -q')
os.system('pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps')

In [ ]:
%%writefile app.py
import ast
import cv2
import os
import numpy as np
import pandas as pd
import plotly.express as px
import urllib.request
import tensorflow_hub as hub
import tensorflow as tf
import streamlit as st
import streamlit.components.v1 as components
import random
import efficientnet.tfkeras as efn
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16, VGG19


st.set_page_config(layout="centered", page_title="APPLE DISEASES DETECTION")
hide_menu_style = """
<style>
footer {visibility: hidden;}
</style>
"""
st.markdown(hide_menu_style, unsafe_allow_html= True)

select_course = st.sidebar.selectbox("Select course", ("CS406.M11", "CS231.M11"))
if select_course == 'CS231.M11':
    select_model = st.sidebar.selectbox("Select model",("EfficientNetB0", "EfficientNetB5", "EfficientNetB7", "EfficientNetV2", "VGG16", "VGG19", "ResNet50-224x224", "ResNet50V2-224x224", "ResNet50-480x480", "ResNet50V2-480x480"))
    preprocess = "None"
else:
    select_model = st.sidebar.selectbox("Select model",("EfficientNetV2-Raw", "EfficientNetV2-Geometric", "EfficientNetV2-Pixel", "EfficientNetV2-Artifacts", "EfficientNetV2-FullAug", "EfficientNetV2-Geo+Artifact"))
    preprocess = st.sidebar.selectbox("Select pre-processing function", ("None", "artifact_preprocess", "geometric_preprocess", "pixel_preprocess", "full_preprocess"))

if select_model == "ResNet50-224x224" or select_model == "ResNet50V2-224x224":
    HEIGHT = 224
    WIDTH = 224
else:
    HEIGHT = 480
    WIDTH = 480
CHANNELS = 3
CLASSES = 6
top_dropout_rate = 0.2
n_label = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']

def load_image(image_id):
    file_path = str(image_id)
    img = cv2.imread(file_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def insect_preprocess(image, p = 0.5, insects_folder='../input/insectdarta/insect'):
    dark_insect=False
    n_insects = random.randint(1, 2)
    aug_prob = random.random()
    flag = False
    if aug_prob < p:
        flag = True
        height, width, _ = image.shape  # target image width and height
        insects_images = os.listdir(insects_folder)
        img_shape = image.shape

        for _ in range(n_insects):
            insect = cv2.cvtColor(cv2.imread(os.path.join(insects_folder, random.choice(insects_images))), cv2.COLOR_RGB2BGR)
            insect = cv2.flip(insect, random.choice([-1, 0, 1]))
            insect = cv2.rotate(insect, random.choice([0, 1, 2]))
            insect = cv2.resize(insect, (width, height))

            h_height, h_width, _ = insect.shape  # insect image width and height
            roi_ho = random.randint(0, image.shape[0] - insect.shape[0])
            roi_wo = random.randint(0, image.shape[1] - insect.shape[1])
            roi = image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            # Creating a mask and inverse mask 
            img2gray = cv2.cvtColor(insect, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            #mask_inv = cv2.cvtColor(cv2.bitwise_not(mask),cv2.COLOR_BGR2GRAY)
            mask_inv = cv2.bitwise_not(mask)

            # Now black-out the area of insect in ROI
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of insect from insect image.
            if dark_insect:
                img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                insect_fg = cv2.bitwise_and(img_bg, img_bg, mask=mask)
            else:
                insect_fg = cv2.bitwise_and(insect, insect, mask=mask)

            # Put insect in ROI and modify the target image
            dst = cv2.add(img_bg, insect_fg, dtype = cv2.CV_64F)

            image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst

    return image
def geometric_preprocess(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)

    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90
        
    # Crops
    if p_crop > .7:
        if p_crop > .9:
            image = tf.image.central_crop(image, central_fraction=.7)
        elif p_crop > .8:
            image = tf.image.central_crop(image, central_fraction=.8)
        else:
            image = tf.image.central_crop(image, central_fraction=.9)
    elif p_crop > .4:
        HEIGHT1 = image.shape[0]
        WIDTH1 = image.shape[1]
        crop_size_h = tf.random.uniform([], int(HEIGHT1*.8), HEIGHT1, dtype=tf.float32)
        crop_size_w = tf.random.uniform([], int(WIDTH1*.8), WIDTH1, dtype=tf.float32)
        image = tf.image.random_crop(image, size=[crop_size_h, crop_size_w, 3])

    return image
def pixel_preprocess(image):
    
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    

    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)

    return image

def full_preprocess(image):
    image = insect_preprocess(image, p = 1.0)
    image = geometric_preprocess(image)
    image = pixel_preprocess(image)
    return image

def process(img, preprocess):
    if preprocess == "None":
        return cv2.resize(img / 255.0, (HEIGHT, WIDTH)).reshape(-1, HEIGHT, WIDTH, 3)
    if preprocess == "full_preprocess":
        img = full_preprocess(img)
    if preprocess == "artifact_preprocess":
        img = insect_preprocess(img, p = 1.0)
    if preprocess == "pixel_preprocess":
        img = pixel_preprocess(img)
    if preprocess == "geometric_preprocess":
        img = geometric_preprocess(img)
    img = tf.cast(img, tf.float32) / 255.0
    img = np.array(img)
    return cv2.resize(img, (HEIGHT, WIDTH)).reshape(-1, HEIGHT, WIDTH, 3)


def get_model_V2():
    hub_url = "../input/efficientnetv2-tfhub-weight-files/tfhub_models/efficientnetv2-l-21k-ft1k/feature_vector"
    
    model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape = [HEIGHT, WIDTH, CHANNELS]),
            hub.KerasLayer(hub_url, trainable = True),
            Dropout(top_dropout_rate, name = "top_dropout"),
            Dense(CLASSES, activation = 'sigmoid')
        ])
    model.build((HEIGHT, WIDTH, CHANNELS))
    
    return model

def get_model_B7():
    weights_path = '../input/d/hunhtnthnh/b7tpu/FGVC8-B7-full_aug.h5'
    base_model = efn.EfficientNetB7(include_top=False, weights='imagenet', input_shape=(HEIGHT, WIDTH, 3), pooling='avg')

    x = base_model.output
    x = Dense(CLASSES, 
        kernel_initializer=tf.keras.initializers.RandomUniform(seed=32),
        bias_initializer=tf.keras.initializers.Zeros(), name='dense_top')(x)
    outputs = Activation('sigmoid', dtype='float32')(x)
    
    model = Model(base_model.input, outputs)
    model.load_weights(weights_path)
    return model

def get_model_B5():
    weights_path = '../input/effnet-b5-model/EfficientNet_B5_Fullaug.h5'
    base_model = efn.EfficientNetB5(include_top=False, weights=None, input_shape=(HEIGHT, WIDTH, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(top_dropout_rate)(x)
    outputs = Dense(CLASSES, activation='sigmoid')(x)
    model = Model(base_model.input, outputs)
    model.load_weights(weights_path)
    return model

def get_model_B0():
    weights_path = '../input/efficientnetb0/FGVC8-efn-b0.h5'
    base_model = efn.EfficientNetB0(include_top=False, weights=None, input_shape=(HEIGHT, WIDTH, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(top_dropout_rate)(x)
    outputs = Dense(CLASSES, activation='sigmoid')(x)
    model = Model(base_model.input, outputs)
    model.load_weights(weights_path)
    return model

def get_model_VGG16():
    weights_path = '../input/fgvc8vgg16/FGVC8-VGG16.h5'
    VGG16_MODEL = VGG16(weights=None ,include_top=False, input_shape=(HEIGHT, WIDTH, 3))
    
    x=VGG16_MODEL.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(256,activation='relu')(x)
    x=Dropout(0.2)(x)
    x=Dense(128,activation='relu')(x)
    prediction=Dense(6,activation='sigmoid')(x)

    model=Model(inputs=VGG16_MODEL.input, outputs=prediction)
    model.load_weights(weights_path)
    return model

def get_model_VGG19():
    weights_path = '../input/fgvc8vgg19/FGVC8-VGG19t.h5'
    VGG19_MODEL = VGG19(weights=None ,include_top=False, input_shape=(HEIGHT, WIDTH, 3))
    
    x=VGG19_MODEL.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(256,activation='relu')(x)
    x=Dropout(0.2)(x)
    x=Dense(128,activation='relu')(x)
    prediction=Dense(6,activation='sigmoid')(x)

    model=Model(inputs=VGG19_MODEL.input, outputs=prediction)
    model.load_weights(weights_path)
    return model

def plot_probs(outputs):
    probs = pd.Series(np.round(outputs * 100, 2), n_label)
    probs = probs.sort_values(ascending=False).reset_index()
    probs.columns = ['Class', 'Probability']
    fig = px.bar(probs, x='Class', y='Probability')
    fig.update_layout(xaxis_tickangle=30)
    st.plotly_chart(fig, use_container_width=True)

st.markdown(
    "<h1 style='text-align: center; color: green;'>IDENTIFY THE CATEGORY OF FOLIAR DISEASES IN APPLE TREES</h1> ",
    unsafe_allow_html=True
    )

logo = plt.imread("../input/logoimage/logo.png")
st.image(logo)

st.markdown(
    "<h1 style='text-align: center;'>Input</h1> ",
    unsafe_allow_html=True
    )

uploaded_file = st.file_uploader("Choose a file")
url = st.text_input(
    'Image Url: ', 
    'https://hort.extension.wisc.edu/files/2014/11/CedarAppleRust_apple.png'
)
st.write('')
st.write('')

if preprocess == "None":
    if uploaded_file is not None:
        bytes_data = uploaded_file.read()
        st.image(bytes_data, use_column_width=True)
        with open('./test.jpg', 'wb') as f: 
            f.write(bytes_data)
    elif url:
        urllib.request.urlretrieve(url, './test.jpg')
        st.markdown(
            f"<center><img src='{url}' style='width: 95%;'></center>",
            unsafe_allow_html=True
        )
    img_test = './test.jpg'
    img = load_image(img_test)
else:
    if uploaded_file is not None:
        bytes_data = uploaded_file.read()
        with open('./test.jpg', 'wb') as f: 
            f.write(bytes_data)
    elif url:
        urllib.request.urlretrieve(url, './test.jpg')
        
    img_test = './test.jpg'
    img = load_image(img_test)
    col1, col2 = st.columns(2)
    with col1:
        st.header("Original Image")
        st.image(img)
    if preprocess == "artifact_preprocess":
        pre_img = insect_preprocess(img, p=1.0)
        with col2:
            st.header("Artifact Preprocess")
            st.image(pre_img)
    if preprocess == "geometric_preprocess":
        pre_img = np.array(geometric_preprocess(img))
        with col2:
            st.header("Geometric Preprocess")
            st.image(pre_img)
    if preprocess == "pixel_preprocess":
        pre_img = np.array(pixel_preprocess(img))
        with col2:
            st.header("Pixel-level Preprocess")
            st.image(pre_img)
    if preprocess == "full_preprocess":
        pre_img = np.array(full_preprocess(img))
        with col2:
            st.header("Full Preprocess")
            st.image(pre_img)

button_predict = st.button("PREDICT")

st.markdown(
    "<h1 style='text-align: center;'>Output</h1> ",
    unsafe_allow_html=True
    )


if button_predict == True:
    if select_model in ["EfficientNetB0", "EfficientNetB5", "EfficientNetB7", "VGG16", "VGG19", "ResNet50-224x224", "ResNet50V2-224x224", "ResNet50-480x480", "ResNet50V2-480x480"]:
        
        if select_model == "EfficientNetB5":
            model = get_model_B5()
            thres = 0.1
        if select_model == "EfficientNetB7":
            model = get_model_B7()
            thres = 0.35
        if select_model == "EfficientNetB0":
            model = get_model_B0()
            thres = 0.3
        if select_model == "VGG16":
            model = get_model_VGG16()
            thres = 0.27
        if select_model == "VGG19":
            model = get_model_VGG19()
            thres = 0.27
        if select_model == "ResNet50-224x224":
            model = load_model("../input/resnet50-sub/ResNet50.h5")
            thres = 0.3
        if select_model == "ResNet50V2-224x224":
            model = load_model("../input/resnet50v2-submit/ResNet50_V2.h5")
            thres = 0.3
        if select_model == "ResNet50-480x480":
            model = load_model("../input/resnet50-480x480-submit/ResNet50_480x480.h5")
            thres = 0.3
        if select_model == "ResNet50V2-480x480":
            model = load_model("../input/resnet50v2-480x480-submit/ResNet50V2_480x480.h5")
            thres = 0.3

        pred = np.array(model.predict(process(img, preprocess)))[0]
        answer = []
        for j in range(len(pred)):
            if pred[j] > thres:
                answer.append(n_label[j])
        if ("healthy" in answer) and (len(answer) > 1):
            answer.remove('healthy')
        if len(answer) == 0:
            answer.append('Can Identify This Image')
        answer = ' '.join(answer)
    elif select_model == "EfficientNetV2-Geo+Artifact":
        model_1 = get_model_V2()
        model_2 = get_model_V2()
        weights_path_1 = '../input/efnet-v2-geoaug-model/EfficientNet_V2_GeoAug.h5'
        weights_path_2 = '../input/efnet-v2-insectaug-model/EfficientNet_V2_InsectAug.h5'
        model_1.load_weights(weights_path_1)
        model_2.load_weights(weights_path_2)
        pred_1 = model_1.layers[2](model_1.layers[1](model_1.layers[0](process(img, preprocess)))).numpy()[0]
        pred_2 = model_2.layers[2](model_2.layers[1](model_2.layers[0](process(img, preprocess)))).numpy()[0]
        pred = (pred_1 + pred_2) / 2
        answer = []
        for j in range(len(pred)):
            if pred[j] > 0.3:
                answer.append(n_label[j])
        if ("healthy" in answer) and (len(answer) > 1):
            answer.remove('healthy')
        if len(answer) == 0:
            answer.append('Can Identify This Image')
        answer = ' '.join(answer)
    else:
        model = get_model_V2()
        if select_model == "EfficientNetV2" or select_model == "EfficientNetV2-FullAug":
            weights_path = '../input/efficientnetv2model/EfficientNet_V2.h5'
            model.load_weights(weights_path)
            thres = 0.2
        if select_model == "EfficientNetV2-Geometric":
            weights_path = '../input/efnet-v2-geoaug-model/EfficientNet_V2_GeoAug.h5'
            model.load_weights(weights_path)
            thres = 0.4
        if select_model == "EfficientNetV2-Pixel":
            weights_path = '../input/efnet-v2-pixelaug-submit/EfficientNet_V2_PixelAug.h5'
            model.load_weights(weights_path)
            thres = 0.2
        if select_model == "EfficientNetV2-Artifacts":
            weights_path = '../input/efnet-v2-insectaug-model/EfficientNet_V2_InsectAug.h5'
            model.load_weights(weights_path)
            thres = 0.2
        if select_model == "EfficientNetV2-Raw":
            weights_path = '../input/efficientnet-v2-raw-model/EfficientNet_V2_raw.h5'
            model.load_weights(weights_path)
            thres = 0.3    
        pred = model.layers[2](model.layers[1](model.layers[0](process(img, preprocess)))).numpy()[0]
        answer = []
        for j in range(len(pred)):
            if pred[j] > thres:
                answer.append(n_label[j])
        if ("healthy" in answer) and (len(answer) > 1):
            answer.remove('healthy')
        if len(answer) == 0:
            answer.append('Can Identify This Image')
        answer = ' '.join(answer)

    st.markdown(f"**Result:** {answer}")  
    plot_probs(pred)




In [ ]:
!ngrok authtoken 21wIJPucxNI3RKbv2bMAgRsw08N_YRH3Z1vGpAGLcSYs5YVs

In [ ]:
from pyngrok import ngrok
 
public_url = ngrok.connect('8501')
public_url

In [ ]:
!streamlit run app.py >/dev/null